# **Anchor Box**

In [27]:
from __future__ import division
import numpy as np
import keras.backend as K
from keras.engine.topology import InputSpec
from keras.engine.topology import Layer


ModuleNotFoundError: ignored

In [29]:
!pip install pyproj
from pyproj import transform


In [30]:
def convert_coordinates(tensor, start_index, conversion, border_pixels='half'):
    '''
    Convert coordinates for axis-aligned 2D boxes between two coordinate formats.
    Creates a copy of `tensor`, i.e. does not operate in place. Currently there are
    three supported coordinate formats that can be converted from and to each other:
        1) (xmin, xmax, ymin, ymax) - the 'minmax' format
        2) (xmin, ymin, xmax, ymax) - the 'corners' format
        2) (cx, cy, w, h) - the 'centroids' format
    Arguments:
        tensor (array): A Numpy nD array containing the four consecutive coordinates
            to be converted somewhere in the last axis.
        start_index (int): The index of the first coordinate in the last axis of `tensor`.
        conversion (str, optional): The conversion direction. Can be 'minmax2centroids',
            'centroids2minmax', 'corners2centroids', 'centroids2corners', 'minmax2corners',
            or 'corners2minmax'.
        border_pixels (str, optional): How to treat the border pixels of the bounding boxes.
            Can be 'include', 'exclude', or 'half'. If 'include', the border pixels belong
            to the boxes. If 'exclude', the border pixels do not belong to the boxes.
            If 'half', then one of each of the two horizontal and vertical borders belong
            to the boxex, but not the other.
    Returns:
        A Numpy nD array, a copy of the input tensor with the converted coordinates
        in place of the original coordinates and the unaltered elements of the original
        tensor elsewhere.
    '''
    if border_pixels == 'half':
        d = 0
    elif border_pixels == 'include':
        d = 1
    elif border_pixels == 'exclude':
        d = -1

    ind = start_index
    tensor1 = np.copy(tensor).astype(np.float)
    if conversion == 'minmax2centroids':
        tensor1[..., ind] = (tensor[..., ind] + tensor[..., ind+1]) / 2.0 # Set cx
        tensor1[..., ind+1] = (tensor[..., ind+2] + tensor[..., ind+3]) / 2.0 # Set cy
        tensor1[..., ind+2] = tensor[..., ind+1] - tensor[..., ind] + d # Set w
        tensor1[..., ind+3] = tensor[..., ind+3] - tensor[..., ind+2] + d # Set h
    elif conversion == 'centroids2minmax':
        tensor1[..., ind] = tensor[..., ind] - tensor[..., ind+2] / 2.0 # Set xmin
        tensor1[..., ind+1] = tensor[..., ind] + tensor[..., ind+2] / 2.0 # Set xmax
        tensor1[..., ind+2] = tensor[..., ind+1] - tensor[..., ind+3] / 2.0 # Set ymin
        tensor1[..., ind+3] = tensor[..., ind+1] + tensor[..., ind+3] / 2.0 # Set ymax
    elif conversion == 'corners2centroids':
        tensor1[..., ind] = (tensor[..., ind] + tensor[..., ind+2]) / 2.0 # Set cx
        tensor1[..., ind+1] = (tensor[..., ind+1] + tensor[..., ind+3]) / 2.0 # Set cy
        tensor1[..., ind+2] = tensor[..., ind+2] - tensor[..., ind] + d # Set w
        tensor1[..., ind+3] = tensor[..., ind+3] - tensor[..., ind+1] + d # Set h
    elif conversion == 'centroids2corners':
        tensor1[..., ind] = tensor[..., ind] - tensor[..., ind+2] / 2.0 # Set xmin
        tensor1[..., ind+1] = tensor[..., ind+1] - tensor[..., ind+3] / 2.0 # Set ymin
        tensor1[..., ind+2] = tensor[..., ind] + tensor[..., ind+2] / 2.0 # Set xmax
        tensor1[..., ind+3] = tensor[..., ind+1] + tensor[..., ind+3] / 2.0 # Set ymax
    elif (conversion == 'minmax2corners') or (conversion == 'corners2minmax'):
        tensor1[..., ind+1] = tensor[..., ind+2]
        tensor1[..., ind+2] = tensor[..., ind+1]
    else:
        raise ValueError("Unexpected conversion value. Supported values are 'minmax2centroids', 'centroids2minmax', 'corners2centroids', 'centroids2corners', 'minmax2corners', and 'corners2minmax'.")

    return tensor1


In [ ]:
class AnchorBoxes(Layer):
  '''
  - Tác dụng: Tạo ra một output tensor chứa tọa độ của các anchor box và các biến
    thể dựa trên input tensor. Một tợp hợp các 2D anchor boxes được tạo ra dựa 
    trên aspect ratios và scale trên mỗi một cells của grid cells. Các hộp được 
    tham số hóa bằng các tọa độ `(xmin, xmax, ymin, ymax)`
    
  - Input shape:
        4D tensor shape `(batch, channels, height, width)` nếu `dim_ordering = 'th'`
        or `(batch, height, width, channels)` nếu `dim_ordering = 'tf'`.

  - Output shape:
        5D tensor of shape `(batch, height, width, n_boxes, 8)`. 
        Chiều cuối cùng gồm 4 tọa độ của anchor box và 4 giá trị biến thể ở mỗi box.
    '''
    def __init__(self,
                img_height,
                img_width,
                this_scale,
                next_scale,
                aspect_ratios = [0.5, 1.0, 2.0],
                two_boxes_for_ar1 = True,
                this_steps = None,
                this_offsets = None,
                clip_boxes = False,
                variances = [0.1, 0.1, 0.2, 0.2],
                coords = 'centroids',
                normalize_coords = False,
                **kwargs):
        '''

        Arguments:
            - img_height (int): chiều cao input images.
            - img_width (int): chiều rộng input images.
            - this_scale (float): một giá trị float thuộc [0, 1], nhân tố scaling kích thước để tạo các anchor boxes dựa trên một tỷ lệ so với cạnh ngắn hơn trong width và height.
            - next_scale (float): giá trị tiếp theo của scale. Được thiết lập khi vào chỉ khi
                `self.two_boxes_for_ar1 == True`.
            - aspect_ratios (list, optional): tợp hợp các aspect ratios của các default boxes được tạo ra từ layer này.
            - two_boxes_for_ar1 (bool, optional): Được sử dụng chỉ khi `aspect_ratios` = 1.
                Nếu `True`, hai default boxes được tạo ra khi aspect ratio = 1. default box đầu tiên sử dụng scaling factor của layer tương ứng,
                default box thứ 2 sử dụng trung bình hình học giữa scaling factor và next scaling factor.
            - clip_boxes (bool, optional): Nếu đúng `True`, giới hạn tọa độ anchor box nằm bên trong hình ảnh.
            - variances (list, optional): Tợp hợp gồm 4 giá trị floats > 0. Là các anchor box offset tương ứng với mỗi tọa độ chia cho giá trị variances tương ứng của nó.
            - coords (str, optional): Tọa độ của box được sử dụng trong model. Có thể là centroids định dạng `(cx, cy, w, h)` (tọa độ box center, width, height),
                hoặc 'corners' định dạng `(xmin, ymin, xmax,  ymax)`, hoặc 'minmax' định dạng `(xmin, xmax, ymin, ymax)`.
            - normalize_coords (bool, optional): Nếu `True` mô hình sử dụng tọa độ tương đối thay vì tuyệt đối. Chẳng hạn mô hình dự đoán tọa độ nằm trong [0, 1] thay vì tọa độ tuyệt đối.
        '''
        if K.backend() != 'tensorflow':
            raise TypeError("This layer inly support TensorFlow at the moment, but you are using {} backend.".format(K.backend()))

        if (this_scale < 0) of (next_scale < 0) of (this_scale > 1):
            raise ValueError("'this_scale' must be [0,1] and 'next_scale' must be > 0, but 'this_scale' == {}, 'next_scale' == {}".format(this_scale, next_scale))

        if len(variances) != 4: 
            raise ValueError("4 variances values must be passed, but {} values were recieved.".format(len(variances)))
        variances = np.array(variances)
        if np.any(variances <= 0):
            raise ValueError("All variances must be > 0, but the variances given are {}".format(variances))
        

        self.img_height = img_height
        self.img_width = img_width
        self.this_scale = this_scale
        self.next_scale = next_scale
        self.aspect_ratios = aspect_ratios
        self.two_boxes_for_ar1 = two_boxes_for_ar1
        self.this_offsets = this_offsets
        self.this_steps = this_steps
        self.clip_boxes = clip_boxes
        self.variances = variances
        self.coords = coords
        self.normalize_coords = normalize_coords
        
        #Tính toán sô lượng box trên 1 cell trường aspect_ratios =1 thì thêm 1 box
        if (1 in aspect_ratios) and two_boxes_for_ar1:
            self.n_boxes = len(aspect_ratios) + 1
        else: 
            self.n_boxes = len(aspect_ratios)
        super(AnchorBoxes, self).__init__(**kwargs)
    

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        super(AnchorBoxes, self).build(input_shape)
    

    def call(self, x, mask = None):
        '''
        Return: Trả về 1 anchor box tensor dựa trên shape của input tensor.

        Tensor này được thiết kế như là hằng số và không tham gia vào quá trình tính toán.

        Arguments:
            x (tensor): 4D tensor có shape `(batch, channels, height, width)` nếu `dim_ordering = 'th'`
                hoặc `(batch, height, width, channels)` nếu `dim_ordering = 'tf'`. Input cho layer này phải là output của các localization predictor layer.
        '''
        #####################################################
        # Bước 1: Tính toán with và heigth của box với mỗi aspect ratio
        #####################################################
        # Cạnh ngẵn hơn của hình ảnh có thể được sử dụng để tính `w` và `h` sử dụng `scale` và `aspect_ratios`.
        size = min(self.img_height, self.img_width)
        #TÍnh toán box (w,h) cho toàn bộ aspect ratios
        wh_list = []
        for ar in slef.aspect_ratios:
            if ar == 1:
                #Tính anchor box thông thường khi aspect_ratio =1.
                box_height = box_width = self.this_scale*size
                wh_list.append((box_width,box_height))
                if self.two_boxes_for_ar1:
                    #Tính version lớn hơn cảu anchor box sử dụng the gometric mean của scale và next scale
                    box_height = box_width = np.sqrt(self.this_scale*next_scale)*size
                    wh_list.append((box_width, box_height))
            else:
                #trường hợp còn lại box_h = scale*size/sqrt(aspect_ratio) và box_w = scale*size*sqrt(aspect_ratio)
                box_width = self.this_scale*size*np.sqrt(ar)
                box_height = self.this_scale*size//np.sqrt(ar)
                wh_list.append((box_width, box_height))
        
        #append vào wh_list
        wh_list = np.array(wh_list)

        #Định hình input_shape
        if K.common.image_dim_ordering() == 'tf':
            batch_size, feature_map_height, feature_map_width, feature_map_channels = x.get_shape().as_list()
        else:
            batch_size, feature_map_channels, feature_map_height, feature_map_width = x.get_shape().as_list()

        
        #Tính các center points của grid of box: CHúng là duy nhất đối với  aspect ratios.
        #########################################################
        #Bước 2: Tính các step size. Khoảng cách là bao xa giữa các anchor box center point theo chiều w và h
        ########################################################
        if (self.this_scale is None):
            step_height = self.img_height // feature_map_height
            step_width = self.img_width // feature_map_width
        else:
            if isinstance(self.this_steps, (list, tuple)) and (len(self.this_steps) == 2):
                step_height = self.this_steps[0]
                strp_width = self.this_steps[1]
            elif isinstance(self.this_steps, (int, float)):
                step_height = self.this_steps
                step_width = self.this_steps
        #Tính toán các offsets cho anchor box conter point đầu tiên từ góc trên cùng bên trái của hình ảnh
        if self.this_offsets is None:
            offset_height = 0.5
            offset_width = 0.5
        else:
            if isinstance(self.this_offsets, (list, tuple)) and (len(self.this_offsets) == 2):
                offset_height = self.this_offsets[0]
                offset_width = self.this_offsets[1]
            elif isinstance(self.this_offsets, (int, float)):
                offset_height self.this_offsets
                offset_width = self.this_offsets
        

        ###############################################################
        # Bước 3: Tính toán các tọa độ của (cx, cy, w, h) theo tọa độ cỉa image gốc.
        ############################################################
        # bây giờ chúng ta có các offsets và step_size, tính grid của anchor box center point
        cx = np.linspace(offset_width*step_width, (offset_width + feature_map_width -1)*step_width, feature_map_width)
        cy = np.linspace(offset_height*step_height, (offset_height + feature_map_height -1)*step_height, feature_map_height)
        cx_grid, cy_grid = np.meshgrid(cx, cy)
        cx_grid = np.expand_dims(cx_grid, -1)
        cy_grid = np.expand_dims(cy_grid, -1)



        #Tạo 1 4D tensor có shape '(feature_map_height, feature_map_width, n_boxes, 4)'
        #Chiều cuối cùng sẽ chứa '(cx, cy, w, h)'

        boxes_tensor = np.zeros((feature_map_height, feature_map_width,, self.n_boxes, 4))

        boxes_tensor[:, :, :, 0] = np.tile(cx_grid, (1, 1, self.n_boxes)) #Đặt cx
        boxes_tensor[:, :, :, 1] = np.tile(cy_grid, (1, 1, self.n_boxes)) #Đặt cy
        boxes_tensor[:, :, :, 2] = wh_list[:, 0] #Đặt w
        boxes_tensor[:, :, :, 3] = wh_list[:, 1] #Đặt h

        #Chuyển '(cx, cy, w, h)' sang '(xmin, xmax, ymin, ymax)'
        boxes_tensor = convert_coordinates(boxes_tensor, start_index=0, conversion='centroids2corners')
